In [12]:
import pandas as pd
import numpy as np

import scipy.io as sc
import scipy.stats
from scipy.fft import fft, fftfreq
from scipy.fftpack import fft, ifft 
from os import listdir
import os.path
import shutil 

import matplotlib.pyplot as plt
import seaborn as sns

# In case you want to remove previously created files

In [13]:
# success = True
# while success == True:
#     try:
#         shutil.rmtree('split_cutting_tests_processed')
#         success = False
#     except:
#         success = False

# Creating new directories for saving the split files 

In [14]:
# # creating directory using os
# success = True
# while success == True:
#     try: 
#         list_of_folders = listdir("cutting_tests_processed")
#         for folder in list_of_folders:
#             os.makedirs(f'split_cutting_tests_processed/{folder}')
#         success = False
#     except:
#         success = False

# Spliting files, Fast Fourrier Transforms and Graphs

In [15]:
# for folder in list_of_folders:
#     list_of_files = listdir(f'cutting_tests_processed/{folder}')
#     for file in list_of_files:
#         if file[0] != 'u':
#             '''
#             loading the original data on MatLab files and creating dataframes 
#             with a time, t, and a distance, y, column. 
#             plots the Fast Fourrier Transform for each dataframea and/or 
#             the original file contents. 
#             saves the FFT outputs into new columns on the dataframe. 
#             spliting each processed data into for new files
#             saves them as files in the same organization logic as the original files
#             files classified as "unknown" presence for chatter have not been used.
#             '''
#             data = sc.loadmat(f'cutting_tests_processed/{folder}/{file}')
#             df = pd.DataFrame.from_dict(data['tsDS'][:,:])
#             df.rename({0: 't', 1: 'y'}, axis=1, inplace=True)
            
#             freq = np.fft.fftfreq(len(df['t']), df['t'][1] - df['t'][0])
#             yf = np.fft.fft(df['y'])
#             df['fft'] = yf
#             df['fftfreq'] = freq # omega
# 
#                 ''' ploting files '''
#             # plt.figure()
#             # plt.title(f'{file}')
#             # plt.xlabel('t')
#             # plt.ylabel('y')
#             # plt.plot(df['t'], df['y'])


#                 ''' ploting FFT results '''
#             # plt.figure()
#             # plt.title(f'{file}')
#             # plt.xlabel('f')
#             # plt.ylabel('A')
#             # plt.plot(freq,yf)
#             # plt.show()

#             df_split = np.array_split(df,4)
#             for split_num in np.arange(4):
#                 df_split[split_num] = df_split[split_num].reset_index(drop=True)
#                 print(df_split[split_num].head())
#                 df_split[split_num].to_csv(f'split_cutting_tests_processed/{folder}/{file[:-4]}_split_{split_num}.csv')


In [16]:
# list_of_folders = listdir("split_cutting_tests_processed")
# for folder in list_of_folders:
#     list_of_files = listdir(f'split_cutting_tests_processed/2inch_stickout')
#     for file in list_of_files:
#         ''' ploting files that were already split with fft transformation
#         '''
#         file_ = pd.read_csv(f'split_cutting_tests_processed/2inch_stickout/{file}')
#         plt.figure(figsize=(10,9))
#         plt.title(f'{file}')
#         plt.xlabel('time')
#         plt.ylabel('FFT amplitude')
#         plt.plot(file_['t'],scipy.fftpack.fft(file_['y'].values))

In [17]:
# list_of_folders = listdir("split_cutting_tests_processed")
# for folder in list_of_folders:
#     list_of_files = listdir(f'split_cutting_tests_processed/{folder}')
#     for file in list_of_files:
#         ''' finding the highest and second highest frequencies
#         these are not saved. 
#         '''
#         file_ = pd.read_csv(f'split_cutting_tests_processed/{folder}/{file}')
#         fft_ = scipy.fftpack.fft(file_['y'].values) 
#         max_fft_amplitude = fft_.max()
#         fft_amplitudes = sorted(np.array(fft_))
#         print(f'file: {file} \n max: {max_fft_amplitude} \n second highest: {fft_amplitudes[-2]}')


# Creating a new data frame with math features 

Defining the functions used in the creation of the dataframe as to keep the formulas more organized and easy to read. 

In [18]:
def thd(data_): # true harmonic distortion
    sq_sum = 0.0
    for r in range(len(data_)):
        sq_sum = sq_sum + data_[r]**2
    sq_harmonics = sq_sum - max(data_)**2
    return 100*sq_harmonics**0.5 / max(data_)

def rms(data_):
    return np.sqrt(np.mean(data_**2))

def amp(data_):
    return max(data_) - min(data_)

def SRA(data_):
    return np.mean(np.sqrt(np.abs(data_)))**2

def crest(data_):
    return max(np.abs(data_))/np.sqrt(np.mean(data_**2))

def impulse(data_):
    return max(np.abs(data_))/np.mean(abs(data_))

def margin(data_):
    return max(abs(data_))/np.mean(np.sqrt(np.abs(data_)))**2

def freq_max(data_):
    # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
    # print(data_)
    print(data_['fftfreq'][data_['fft'] == data_['fft'].max()].values[0])

def freq_sec_highest(data_):
    # sorted(np.array(scipy.fftpack.fft(file['fftfreq'].values)))[-3]
    data_sorted = data_.sort_values(by = ['fft'], ascending = False)
    return data_sorted['fftfreq'][1]

def freq_mean(data_):
    return data_['fftfreq'].mean()

# def freq_pico(data_):
#     # sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]
#     return data_['fftfreq'].max()

#         cols['freq_max'].append(scipy.fftpack.fft(file['y'].values).max()) # https://gist.github.com/endolith/255291
#         cols['freq_sec_highest'].append(sorted(np.array(scipy.fftpack.fft(file['y'].values)))[-2]) 
#         cols['freq_mean'].append(scipy.fftpack.fft(file['y'].values).mean()) 


In [20]:
''' Creating the data frame of math equations outputs.
'''
cols = {'Experiment': [],
        'chatter_presence': [],
        'med': [],
        'std': [],
        'rms': [],
        'curtose': [],
        'distortion': [],
        'amplitude': [],
        'sra': [],
        'crest': [],
        'impulse': [],
        'margin': [],
        'max': [],
        'freq_max': [],
        'freq_sec_highest': [],
        'freq_mean': []}

list_of_folders = listdir("corrected")
for folder in list_of_folders:
    list_of_files = listdir(f'corrected/{folder}')
    for file in list_of_files:
        cols['Experiment'].append(f'{file[:-4]}')
        cols['chatter_presence'].append(f'{file[:1]}')
        
        # print(file)
        file = pd.read_csv(f'corrected/{folder}/{file}', index_col=0)
                
        cols['med'].append(file['y'].mean())
        cols['std'].append(file['y'].std())
        cols['rms'].append(rms(file['y']))
        
        cols['curtose'].append(scipy.stats.kurtosis(file['y']))
        cols['distortion'].append(thd(file['y']))
        cols['amplitude'].append(amp(file['y']))
        cols['sra'].append(SRA(file['y']))
        
        cols['crest'].append(crest(file['y']))
        cols['impulse'].append(impulse(file['y']))
        cols['margin'].append(margin(file['y']))
        cols['max'].append(file['y'].max())
        
        cols['freq_max'].append(freq_max(file)) # https://gist.github.com/endolith/255291
        cols['freq_sec_highest'].append(freq_sec_highest(file)) 
        cols['freq_mean'].append(freq_mean(file))

dataframe = pd.DataFrame(cols)

c_770_005_split_3.csv
-945.8624628315024
c_770_002_split_0.csv
933.5751046660512
s_570_005_split_2.csv
-2623.422929243635
s_425_025_split_1.csv
2905.48378062974
s_320_010_split_2_noise_added.csv
-2874.400000006699
c_425_025_split_0_noise_added.csv
863.2581952661802
c_320_005_split_3_noise_added.csv
-860.8000000020062
c_770_010_split_3.csv
-845.3566185830042
s_425_025_split_0.csv
612.375504975789
c_425_025_split_3_noise_added.csv
-939.7333333355234
s_770_002_2_split_0.csv
1071.7785644312094
c_770_010_split_3_noise_added.csv
920.9600000021464
s_425_020_split_3_noise_added.csv
2348.800000005474
c_770_002_split_0_noise_added.csv
-970.5631399296924
c_570_002_split_1_noise_added.csv
-1757.5699684668505
s_570_002_split_1.csv
3556.252399854698
c_770_001_split_3.csv
-908.7818243656308
s_425_020_split_3.csv
-884.9215343894637
c_770_001_split_3_noise_added.csv
-847.2000000019744
c_770_010_split_2.csv
-4018.2639269893466
c_425_025_split_0.csv
918.658500815467
c_320_005_split_2_noise_added.csv
-482

In [21]:
dataframe

,Experiment,chatter_presence,med,std,rms,curtose,distortion,amplitude,sra,crest,impulse,margin,max,freq_max,freq_sec_highest,freq_mean
0,c_770_005_split_3,c,0.005414,0.040638,0.040996,0.898666,4733.248435,0.416909,0.026842,5.159852,6.625808,7.880688,0.205375,None,-2499.944445,-1250.016667
1,c_770_002_split_0,c,0.005469,0.014140,0.015160,3.312443,1899.488742,0.160458,0.008789,5.493718,7.627771,9.475498,0.077174,None,0.266660,1249.966668
2,s_570_005_split_2,s,0.005490,0.003661,0.006599,0.890928,2356.555154,0.030300,0.005234,3.135348,3.592701,3.952511,0.020689,None,-4999.542878,-3750.228561
3,s_425_025_split_1,s,0.005513,0.002263,0.005959,-0.401359,3478.401595,0.015441,0.005257,2.271856,2.452552,2.575449,0.013538,None,2500.699972,3750.049998
4,s_320_010_split_2_noise_added,s,0.005502,0.003068,0.006299,0.242878,3374.119086,0.029528,0.005119,3.312104,3.737711,4.075462,0.020864,None,0.800000,-0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,s_570_014_split_0_noise_added,s,0.005456,0.001732,0.005724,-0.279883,5052.619381,0.012987,0.005306,2.069509,2.171051,2.232369,0.011846,None,0.914244,-0.457122
284,s_770_005_split_2,s,0.005482,0.003982,0.006776,1.860644,3536.756650,0.037606,0.005365,3.388638,3.899269,4.279893,0.022961,None,-4999.739135,-3750.021739
285,s_570_012_split_2,s,0.005484,0.001980,0.005830,-0.255692,10000.500110,0.016045,0.005283,2.235844,2.376338,2.467320,0.013036,None,-4999.925000,-3750.006250
286,c_770_005_split_3_noise_added,c,0.005426,0.008870,0.010397,1.034704,1766.022171,0.087902,0.007025,4.583835,5.757399,6.784075,0.040242,None,2.133561,0.000000
